<a href="https://colab.research.google.com/github/SupunGurusinghe/customer-loyalty-ML/blob/main/customer_loyalty_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Imports**

In [3]:
!pip install gower

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gower: filename=gower-0.0.5-py3-none-any.whl size=4232 sha256=2e70314a26227d9eda990df21f63c8c97b290c3c6e8c37e9bf9220317fdf97ee
  Stored in directory: /root/.cache/pip/wheels/3e/f9/9a/67122a959a424e9cbb4557a8366c871a30e31cd75f0d003db4
Successfully built gower


In [21]:
import pandas as pd
import numpy as np
import gower
import datetime as dt
from sklearn.cluster import DBSCAN


In [5]:
url = 'https://raw.githubusercontent.com/SupunGurusinghe/customer-loyalty-ML/main/superstore.csv'
df = pd.read_csv(url ,encoding='windows-1252')
df.head()

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [6]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'], infer_datetime_format=True)
df['ShipDate'] = pd.to_datetime(df['ShipDate'], infer_datetime_format=True)
df.head()

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [7]:


least_recent_date = df['OrderDate'].min()
most_recent_date = df['OrderDate'].max()

print(f'First day: {least_recent_date} \nLast day: {most_recent_date}')

First day: 2014-01-03 00:00:00 
Last day: 2017-12-30 00:00:00


In [8]:
df = df[df['OrderDate'].dt.year == 2017]
df.shape[0]

3312

In [9]:
df.head()

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
12,13,CA-2017-114412,2017-04-15,2017-04-20,Standard Class,AA-10480,Andrew Allen,Consumer,United States,Concord,...,28027,South,OFF-PA-10002365,Office Supplies,Paper,Xerox 1967,15.552,3,0.2,5.4432
23,24,US-2017-156909,2017-07-16,2017-07-18,Second Class,SF-20065,Sandra Flanagan,Consumer,United States,Philadelphia,...,19140,East,FUR-CH-10002774,Furniture,Chairs,"Global Deluxe Stacking Chair, Gray",71.372,2,0.3,-1.0196
34,35,CA-2017-107727,2017-10-19,2017-10-23,Second Class,MA-17560,Matt Abelman,Home Office,United States,Houston,...,77095,Central,OFF-PA-10000249,Office Supplies,Paper,Easy-staple paper,29.472,3,0.2,9.9468
41,42,CA-2017-120999,2017-09-10,2017-09-15,Standard Class,LC-16930,Linda Cazamias,Corporate,United States,Naperville,...,60540,Central,TEC-PH-10004093,Technology,Phones,Panasonic Kx-TS550,147.168,4,0.2,16.5564
43,44,CA-2017-139619,2017-09-19,2017-09-23,Standard Class,ES-14080,Erin Smith,Corporate,United States,Melbourne,...,32935,South,OFF-ST-10003282,Office Supplies,Storage,"Advantus 10-Drawer Portable Organizer, Chrome ...",95.616,2,0.2,9.5616


In [10]:
# Recency

date = '2018-01-01'
recency = pd.to_datetime(date).normalize() - df.groupby('CustomerID').agg(MaximumDate=('OrderDate', np.max))
recency = recency.astype(str).replace(r'\D+', '', regex=True)
print(recency)

           MaximumDate
CustomerID            
AA-10315           186
AA-10375            21
AA-10480           261
AA-10645            57
AB-10060            56
...                ...
XP-21865            45
YC-21895             6
YS-21880            11
ZC-21910            56
ZD-21925           204

[693 rows x 1 columns]


In [11]:
# Frequency

frequency = df[pd.to_datetime(df['OrderDate']).dt.month == 12].groupby('CustomerID')['RowID'].count()
print(frequency)

CustomerID
AA-10375    3
AB-10165    2
AC-10450    1
AC-10615    1
AD-10180    2
           ..
VF-21715    1
VW-21775    3
WB-21850    2
YC-21895    1
YS-21880    1
Name: RowID, Length: 195, dtype: int64


In [12]:
# Monetary

monetary = df[pd.to_datetime(df['OrderDate']).dt.month == 12].groupby('CustomerID')['Sales'].sum()
print(monetary)

CustomerID
AA-10375     149.872
AB-10165      39.792
AC-10450    1665.620
AC-10615      37.930
AD-10180     916.590
              ...   
VF-21715      28.160
VW-21775     678.100
WB-21850      91.193
YC-21895     750.680
YS-21880    1000.020
Name: Sales, Length: 195, dtype: float64


In [17]:
f_df = recency.join(frequency).join(monetary).rename(columns={'RowID':'Count'}).reset_index()
convert_dict = {'CustomerID': str,
                'MaximumDate': int,
                'Count': float,
                'Sales': float
                }
 
f_df = f_df.astype(convert_dict)
f_df = f_df.fillna(0)
f_df

,CustomerID,MaximumDate,Count,Sales
0,AA-10315,186,0.0,0.000
1,AA-10375,21,3.0,149.872
2,AA-10480,261,0.0,0.000
3,AA-10645,57,0.0,0.000
4,AB-10060,56,0.0,0.000
...,...,...,...,...
688,XP-21865,45,0.0,0.000
689,YC-21895,6,1.0,750.680
690,YS-21880,11,1.0,1000.020
691,ZC-21910,56,0.0,0.000


In [18]:
distance_matrix = gower.gower_matrix(f_df)

In [26]:

# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.3, 
                        min_samples=2, 
                        metric="precomputed")

# Fitting the clustering algorithm
dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
f_df["cluster"] = dbscan_cluster.labels_

In [27]:
f_df

,CustomerID,MaximumDate,Count,Sales,cluster
0,AA-10315,186,0.0,0.000,0
1,AA-10375,21,3.0,149.872,0
2,AA-10480,261,0.0,0.000,0
3,AA-10645,57,0.0,0.000,0
4,AB-10060,56,0.0,0.000,0
...,...,...,...,...,...
688,XP-21865,45,0.0,0.000,0
689,YC-21895,6,1.0,750.680,0
690,YS-21880,11,1.0,1000.020,0
691,ZC-21910,56,0.0,0.000,0


In [31]:
f_df.cluster.unique()

array([ 0, -1])

In [34]:
# selecting rows based on condition
rslt_df = f_df.loc[f_df['cluster'] < 0]
  
print('\nResult dataframe :\n', rslt_df)


Result dataframe :
     CustomerID  MaximumDate  Count     Sales  cluster
181   DJ-13630           14    4.0  2888.518       -1
379   KH-16510           24    9.0  3955.507       -1
564   RM-19675            8    2.0  2970.432       -1
660   TP-21130           10    7.0  2834.980       -1
